In [2]:
from dotenv import load_dotenv
import os, json
import requests

load_dotenv()
api_key = os.getenv("API_KEY", None)
project_id = os.getenv("PROJECT_ID", None)
ibm_cloud_iam_url = os.getenv("IAM_IBM_CLOUD_URL", None)

creds = {
    "url"    : "https://us-south.ml.cloud.ibm.com",
    "apikey" : api_key
}

params = {
    "decoding_method":"greedy",
    "max_new_tokens":3000,
    "min_new_tokens":1,
    # "temperature":0.1,
    "top_k":50,
    "top_p":1,
     "repetition_penalty": 1.1,
    "stop_sequences":["```","<|user|>","<|assistant|>"],
}

In [3]:
# Prepare the payload and headers
payload = {
    "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
    "apikey": api_key
}
headers = {
    'Content-Type': "application/x-www-form-urlencoded"
}

# Make a POST request while ignoring SSL certificate verification
try:
    response = requests.post(f"https://{ibm_cloud_iam_url}/identity/token", data=payload, headers=headers, verify=False)
    
    # Check if the request was successful
    response.raise_for_status()

    # Parse the JSON response
    decoded_json = response.json()
    access_token = decoded_json["access_token"]
    # print(f"Access Token: {access_token}")
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'iam.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [6]:
import requests

GRANITE_3B_CODE_INSTRUCT = "ibm/granite-3b-code-instruct"
GRANITE_8B_CODE_INSTRUCT = "ibm/granite-8b-code-instruct"
GRANITE_20B_CODE_INSTRUCT = "ibm/granite-20b-code-instruct"
GRANITE_34B_CODE_INSTRUCT = "ibm/granite-34b-code-instruct"
GRANITE_3_8B_INSTRUCT = "ibm/granite-3-8b-instruct"

def watsonx_code_gen(prompt,model_id):

    url = "https://us-south.ml.cloud.ibm.com/ml/v1/text/generation_stream?version=2023-05-29"

    body = {
        "input": prompt,
        "parameters": params,
        "model_id": model_id,
        "project_id": project_id
    }

    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.post(
        url,
        headers=headers,
        json=body,
        stream=True
    )

    if response.status_code != 200:
        raise Exception("Non-200 response: " + str(response.text))

    answer = ""
    # Stream the response
    for line in response.iter_lines():
        if line:  # Ensure the line is not empty
            decoded_line = line.decode("utf-8").strip()
            
            # Check if the line starts with "data: "
            if decoded_line.startswith("data: "):
                json_data = decoded_line[len("data: "):]  # Remove the "data: " prefix
                
                try:
                    # Attempt to load the JSON data
                    data = json.loads(json_data)
                    generated_text = data.get("results", "")
                    answer += generated_text[0]["generated_text"]
                    # print(generated_text[0]["generated_text"],end="")  # Print or process the generated text as needed
                except json.JSONDecodeError:
                    print("Failed to decode JSON:", json_data)
                except Exception as e:
                    print("An error occurred:", e)

    return answer


In [7]:
sample_code = """import java.util.*;

public class Main {

    public static void main(String[] args) {
        try (Scanner in = new Scanner(System.in)) {
            int n = in.nextInt();
            int[] a = new int[n];
            for(int a_i=0; a_i < n; a_i++){
                a[a_i] = in.nextInt();
            }
        }
    }

    public void calculate(int[] a) {
        int sum = 0;
        for (int i = 0; i < a.length; i++) {
            sum += a[i];
        }
        System.out.println(sum);
    }

    public void print(int[] a) {
        for (int i = 0; i < a.length; i++) {
            System.out.println(a[i]);
        }
    }

    public void sort(int[] a) {
        Arrays.sort(a);
        double median;
        if (a.length % 2 == 0) {
            median = (a[a.length / 2 - 1] + a[a.length / 2]) / 2.0;
        } else {
            median = a[a.length / 2];
        }
    }
}"""

bad_code = """import java.util.*;

public class Main {

    public static void main(String[] args) {
        try (Scanner in = new Scanner(System.in)) {
            int n = in.nextInt();
    }

    public void calculate(int[] a) {
        int sum = 0;
        for (int i = 0; 
        }
        System.out.println(sum);
    }

    public void print(int[] a) {
        for (int i = 0; System.out.println(a[i]);
        }
    }

    public void sort(int[] a) {
        Arrays.sort(a);
        double me
        if (a.length % 2 == 0) {
           th / 2 - 1] + a[a.length / 2]) / 2.0;
        } else {
            median = a[a.len
        }
    }
}"""

In [8]:
model_id = GRANITE_3_8B_INSTRUCT
language = "java"
documentframework = "javadoc"
testframework = "junit"

generate javadoc

In [28]:
prompt = f"""<|user|>
generate {documentframework} document for the {language} provided.
-generate the {language} code only.
{language}: `{sample_code}`
<|assistant|>
```{language}"""

answer = watsonx_code_gen(prompt,model_id).replace("```","")
print(answer)


/**
 * This class represents the main application that reads an array of integers from the user,
 * calculates the sum of its elements, prints each element, sorts the array, and finds the median.
 */
public class Main {

    /**
     * The main method that reads an array of integers from the user, calculates the sum of its elements,
     * prints each element, sorts the array, and finds the median.
     * @param args command line arguments
     */
    public static void main(String[] args) {
        try (Scanner in = new Scanner(System.in)) {
            int n = in.nextInt(); // read the number of elements in the array
            int[] a = new int[n]; // create an array to store the elements
            for(int a_i=0; a_i < n; a_i++){ // read the elements of the array
                a[a_i] = in.nextInt();
            }
        }
    }

    /**
     * Calculates the sum of the elements in the given array.
     * @param a the array of integers
     */
    public void calculate(int[] a

score, review and improve

In [41]:
prompt = f"""<|user|>
you are {language} expert. you need to review the following {language} code. 
-please understand the {language} code provided in detail.
-please review the code in completeness, logic, performance and quality.
-make it more object oriented.
-give a score out of 10 for the quality of code.
-please provide constructive suggestion in detail.
-generate the review and suggestion only.
{language} code: `{sample_code}`
-DONT generate the improve code directly.
<|assistant|>
score and review:"""

suggestion = watsonx_code_gen(prompt,model_id)
print(suggestion)


The provided Java code is a simple program that reads an array of integers from the user, calculates the sum of its elements, prints each element on a new line, sorts the array, and then calculates and prints the median value. The code is functional and follows basic Java syntax and conventions. However, there are several areas where the code could be improved in terms of object orientation, readability, and maintainability.

Here's a breakdown of the code:

1. The `Main` class contains the `main` method, which is the entry point of the program. It reads an integer `n` from the user, creates an array of integers `a` of size `n`, and then calls the `calculate`, `print`, and `sort` methods on this array.
2. The `calculate` method calculates the sum of all elements in the array and prints it.
3. The `print` method prints each element of the array on a new line.
4. The `sort` method sorts the array using the `Arrays.sort()` method and then calculates and prints the median value. If the ar

In [45]:
prompt = f"""<|user|>
improve the {language} provided base on the review provided.
-understand the review suggestion in detail.
-adopt the review suggestion if applicable.
###
original {language} code: `{sample_code}`
review suggestion: `{suggestion}`
<|assistant|>
improved {language}
```{language}"""

answer = watsonx_code_gen(prompt,model_id).replace("```","")
print(answer)


import java.util.*;

public class ArrayOperations {

    private int[] numbers;

    public ArrayOperations(int[] numbers) {
        this.numbers = numbers;
    }

    public double calculateSum() {
        int sum = 0;
        for (int number : numbers) {
            sum += number;
        }
        return sum;
    }

    public void printNumbers() {
        for (int number : numbers) {
            System.out.println(number);
        }
    }

    public void sortNumbers() {
        Arrays.sort(numbers);
    }

    public double calculateMedian() {
        int size = numbers.length;
        if (size % 2 == 0) {
            return (numbers[size / 2 - 1] + numbers[size / 2]) / 2.0;
        } else {
            return numbers[size / 2];
        }
    }
}

public class Main {

    public static void main(String[] args) {
        try (Scanner in = new Scanner(System.in)) {
            int n = in.nextInt();
            if (n <= 0) {
                System.out.println("Error: Number of eleme

fix bad code

In [62]:
prompt = f"""<|user|>
fix the {language} provided.
-understand the review suggestion in detail.
-identify if any problem.
-fix the problem if any.
-add explanation as inline comment.

###
{language}: `{bad_code}`
<|assistant|>
fixed version with inline comment at the point it fix.
```{language}"""

answer = watsonx_code_gen(prompt,model_id).replace("```","")
print(answer)


import java.util.*;

public class Main {

    public static void main(String[] args) {
        try (Scanner in = new Scanner(System.in)) {
            int n = in.nextInt();
            int[] a = new int[n];
            for (int i = 0; i < n; i++) {
                a[i] = in.nextInt();
            }
            sort(a);
            calculate(a);
            print(a);
        }
    }

    public void calculate(int[] a) {
        int sum = 0;
        for (int i = 0; i < a.length; i++) { // fixed: loop condition
            sum += a[i];
        }
        System.out.println(sum);
    }

    public void print(int[] a) {
        for (int i = 0; i < a.length; i++) { // fixed: loop condition
            System.out.println(a[i]);
        }
    }

    public void sort(int[] a) {
        Arrays.sort(a);
        double median;
        if (a.length % 2 == 0) { // fixed: corrected condition
            median = (a[a.length / 2 - 1] + a[a.length / 2]) / 2.0;
        } else {
            median = a[a.

generate diagram

In [66]:
prompt = f"""<|user|>
generate diagram for the {language} code provided.
-understand the review suggestion in detail.
-use simple name for node.
-generate in mermaid.
-dont repeating.
{language}: `{sample_code}`
<|assistant|>
```mermaid"""

answer = watsonx_code_gen(prompt,model_id).replace("```","")
print(answer)


graph TD
    A[Main] --> B[main]
    B --> C[try-with-resources]
    C --> D[Scanner in]
    D --> E[nextInt]
    E --> F[n]
    F --> G[new int[n]]
    G --> H[calculate]
    H --> I[sum]
    I --> J[for loop]
    J --> K[a[i]]
    K --> L[sum += a[i]]
    M[print] --> N[for loop]
    N --> O[a[i]]
    P[sort] --> Q[Arrays.sort]
    Q --> R[if-else]
    R --> S[a.length % 2 == 0]
    S --> T[median = (a[a.length / 2 - 1] + a[a.length / 2]) / 2.0]
    U[else] --> V[median = a[a.length / 2]]



conversion

In [69]:
target_language = "python"

prompt = f"""<|user|>
rewrite the {language} code provided to {target_language}.
-understand the code provided in detail.
-understand how to map technology and syntax to the target {target_language} langauge.
{language}: `{sample_code}`
<|assistant|>
```{target_language}"""

answer = watsonx_code_gen(prompt,model_id).replace("```","")
print(answer)


import array as arr

def main():
    n = int(input())
    a = arr.array('i', [0] * n)
    for i in range(n):
        a[i] = int(input())
    calculate(a)
    sort(a)
    print(median(a))

def calculate(a):
    sum = 0
    for i in range(len(a)):
        sum += a[i]
    print(sum)

def sort(a):
    a.sort()
    if len(a) % 2 == 0:
        median = (a[len(a) // 2 - 1] + a[len(a) // 2]) / 2.0
    else:
        median = a[len(a) // 2]
    print(median)

def median(a):
    return sort(a)[len(a) // 2]

if __name__ == "__main__":
    main()



generate api definition

In [11]:
prompt = f"""<|user|>
generate a API definition in swagger for the {language} code provided.
{language}: `{sample_code}`
<|assistant|>
```yaml
"""

answer = watsonx_code_gen(prompt,model_id).replace("```","")
print(answer)

openapi: 3.0.0
info:
  title: Java Code API
  version: 1.0.0
servers:
  - url: http://api.example.com/java-code
paths:
  /calculate:
    post:
      summary: Calculate the sum of an array
      requestBody:
        content:
          application/json:
            schema:
              type: array
              items:
                type: integer
        required: true
      responses:
        '200':
          description: The sum of the array
          content:
            application/json:
              schema:
                type: number
  /print:
    post:
      summary: Print the elements of an array
      requestBody:
        content:
          application/json:
            schema:
              type: array
              items:
                type: integer
        required: true
      responses:
        '200':
          description: The elements of the array printed to the console
  /sort:
    post:
      summary: Sort an array and return the median
      requestBody:
        c

generate test case

In [12]:
prompt = f"""<|user|>
generate Test Cases in {testframework} for the {language} code provided.
-focus on boundary cases.
-focus on exception handling
{language}: `{sample_code}`
<|assistant|>
```java"""

answer = watsonx_code_gen(prompt,model_id).replace("```","")
print(answer)


import org.junit.jupiter.api.Test;
import static org.junit.jupiter.api.Assertions.*;
import java.io.ByteArrayInputStream;
import java.io.ByteArrayOutputStream;
import java.io.PrintStream;
import java.util.Arrays;

class MainTest {

    @Test
    void testCalculate() {
        Main main = new Main();
        int[] a = {1, 2, 3, 4, 5};
        assertEquals(15, main.calculate(a));

        a = new int[]{Integer.MAX_VALUE, Integer.MIN_VALUE};
        assertEquals(-2147483647, main.calculate(a));

        a = new int[]{0};
        assertEquals(0, main.calculate(a));

        a = new int[]{};
        assertEquals(0, main.calculate(a));
    }

    @Test
    void testPrint() {
        ByteArrayOutputStream out = new ByteArrayOutputStream();
        PrintStream originalOut = System.out;
        System.setOut(new PrintStream(out));

        Main main = new Main();
        int[] a = {1, 2, 3, 4, 5};
        main.print(a);

        String output = out.toString();
        assertEquals("1\n2\n3\n4\

generate explanation

In [14]:
prompt = f"""<|user|>
Please provide a detailed explanation for the following {language} code:
-ensure have new line after every full stop.

###
code: `{sample_code}`
<|assistant|>
explanation:
"""

answer = watsonx_code_gen(prompt,model_id)
print(answer)


This Java code is a simple program that reads an array of integers from the user, calculates the sum of its elements, prints each element on a new line, sorts the array, and then finds and prints the median value. Here's a step-by-step explanation:

1. The `main` method initializes a `Scanner` object to read input from the user. It then reads an integer `n`, which represents the size of the array, and creates an integer array `a` of size `n`.

2. The `calculate` method takes the array as input and calculates the sum of its elements by iterating through the array and adding each element to a running total. It then prints the sum.

3. The `print` method takes the array as input and prints each element on a new line using a `for` loop.

4. The `sort` method takes the array as input and sorts it in ascending order using the `Arrays.sort()` method. It then calculates the median value based on whether the array has an even or odd number of elements. If the array has an even number of elemen